In [9]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib

class LinearRegressionScratch:
    def __init__(self, learning_rate=0.01, epochs=500):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.W = None
        self.b = None
        self.scaler_X = StandardScaler()
        self.scaler_y = StandardScaler()

    def _mse_loss(self, y_pred, y_true):
        return torch.mean((y_pred - y_true) ** 2)

    def fit(self, csv_file):
        # Load data
        df = pd.read_csv(csv_file)
        X = df.iloc[:, :-1].values
        y = df.iloc[:, -1].values.reshape(-1, 1)

        # Scale
        X_scaled = self.scaler_X.fit_transform(X)
        y_scaled = self.scaler_y.fit_transform(y)

        # Train/test split
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y_scaled, test_size=0.2, random_state=42
        )

        # Convert to torch tensors
        X_train = torch.tensor(X_train, dtype=torch.float32)
        y_train = torch.tensor(y_train, dtype=torch.float32)

        num_features = X_train.shape[1]

        # Initialize weights and bias
        self.W = torch.randn((num_features, 1), requires_grad=True)
        self.b = torch.zeros((1,), requires_grad=True)

        # Training loop
        for epoch in range(self.epochs):
            y_pred = X_train @ self.W + self.b
            loss = self._mse_loss(y_pred, y_train)

            loss.backward()

            with torch.no_grad():
                self.W -= self.learning_rate * self.W.grad
                self.b -= self.learning_rate * self.b.grad

            self.W.grad.zero_()
            self.b.grad.zero_()

            if (epoch + 1) % 50 == 0:
                print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}")

        # Save scalers & params
        joblib.dump(self.scaler_X, "scaler_X.pkl")
        joblib.dump(self.scaler_y, "scaler_y.pkl")
        torch.save({"W": self.W, "b": self.b}, "linear_params.pt")

    def predict(self, X_new):
        if self.W is None or self.b is None:
            # Load saved model if available
            checkpoint = torch.load("linear_params.pt")
            self.W = checkpoint["W"]
            self.b = checkpoint["b"]
            self.scaler_X = joblib.load("scaler_X.pkl")
            self.scaler_y = joblib.load("scaler_y.pkl")

        # Scale input
        X_scaled = self.scaler_X.transform(np.array(X_new).reshape(1, -1))
        X_tensor = torch.tensor(X_scaled, dtype=torch.float32)

        # Prediction
        y_pred_scaled = X_tensor @ self.W + self.b
        y_pred = self.scaler_y.inverse_transform(y_pred_scaled.detach().numpy())

        return y_pred[0][0]


In [7]:
# Create fake data
num_samples = 100
num_features = 5
X_fake = np.random.rand(num_samples, num_features) * 100
y_fake = X_fake[:, 0] * 2 + X_fake[:, 1] * 5 + np.random.randn(num_samples) * 10
fake_data = np.hstack((X_fake, y_fake.reshape(-1, 1)))

df_fake = pd.DataFrame(fake_data, columns=[f'feature_{i+1}' for i in range(num_features)] + ['target'])
fake_csv_path = '/tmp/fake_data.csv'
df_fake.to_csv(fake_csv_path, index=False)
print(f"Saved fake data to {fake_csv_path}")


Saved fake data to /tmp/fake_data.csv


In [11]:
# Train the model
model = LinearRegressionScratch(learning_rate=0.01, epochs=500)
model.fit(fake_csv_path)

# Predict on new data
sample_features = [5.1, 3.5, 1.4, 0.2, 2.0]  # Example input
print("Prediction:", model.predict(sample_features))


Epoch 50: Loss = 0.7981
Epoch 100: Loss = 0.1956
Epoch 150: Loss = 0.0533
Epoch 200: Loss = 0.0174
Epoch 250: Loss = 0.0082
Epoch 300: Loss = 0.0058
Epoch 350: Loss = 0.0052
Epoch 400: Loss = 0.0050
Epoch 450: Loss = 0.0050
Epoch 500: Loss = 0.0050
Prediction: 25.354132
